## Код, который не относится к проблеме переведен в `raw`, чтобы не запускаться.

In [1]:
from tqdm import tqdm
from typing import Dict
import time

import torch
import torch.fx
from torch.fx.node import Node

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import module2graph

Тут был код, который замерял время. Сейчас он в файле

Научимся семлпировать нужным образом

In [2]:
class GraphInterperterWithGumbelSoftmaxGamma(module2graph.GraphInterperterWithGamma):
    def __init__(self, mod, gamma_shift=0.0, temperature=1.0):
        self.gamma_shift = gamma_shift
        self.temperature = temperature
        super().__init__(mod)

    def init_gammas(self):
        i = 0
        gammas = []
        self.gammas_name = {}
        for node in self.graph.nodes:
            if node.op == 'call_module':
                gammas.append(np.random.randn()+self.gamma_shift)
                self.gammas_name[str(node)] = i# перевод в str тут для удобства. в реалньых методах это не нужно
                i+=1                        # да и вообще, тут по идее должен быть тензор/параметр
        self.gammas =  torch.nn.Parameter(torch.as_tensor(gammas), requires_grad = True)
        self.discrete = False 

    def sample_gammas(self):
        if self.discrete:
            return self.gammas
        else:
            d = torch.distributions.RelaxedBernoulli(logits=self.gammas, temperature=self.temperature)
            return d.rsample()
        
    def make_gammas_discrete(self):
        self.gammas.data = (self.gammas.data>=0) * 1.0
        self.gammas.requires_grad = False 
        self.discrete = True

Получим претрейн на n эпох

In [3]:
NUM_EPOCHS=8

%cd fast_nas_adapt/cv_experiment
!python3 run.py --strategy=aggressive --log_dir=results/aggressive \
                --num_epochs={NUM_EPOCHS} \
                --device=cpu --lr=5e-2 --weight_decay=5e-4 \
                --classes 0 1 2 3 4 5 6 7 8 9

/Users/b1/Desktop/m1p/2023-Problem-140/code/basic_experiment/fast_nas_adapt/cv_experiment
ExpConfig(img_size=33, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], mse_weight=0.1, strategy='aggressive', batch_size=128, num_epochs=8, log_dir='results/aggressive', device='cpu', lr=0.05, momentum=0.9, weight_decay=0.0005, use_scheduler=False)
Using cache found in /Users/b1/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weigh

Замерим время работы

In [4]:
from model import ResNet18
from data import get_dataloaders
from trainer import Trainer

In [5]:
from config import ExpConfig
config = ExpConfig()
config.img_size = 33
config.classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
config.model_path = f'results/aggressive/model_{NUM_EPOCHS}.ckpt'

config.mse_weight = 0.1

config.strategy = 'aggressive'
config.batch_size = 64
config.num_epochs = 0
config.log_dir = 'results/aggressive'
config.device = 'cpu'

config.lr = 5e-2
config.momentum = 0.9
config.weight_decay = 5e-4
config.use_scheduler = False

In [6]:
model = ResNet18(num_classes=len(config.classes))
model_state_dict = torch.load(config.model_path, map_location='cpu')
model_state_dict = {key: model_state_dict[key] for key in model_state_dict if 'fc' not in key}
model.model.load_state_dict(model_state_dict, strict=False)
train_dl, test_dl = get_dataloaders(classes=config.classes, batch_size=config.batch_size,
                                    img_size=config.img_size)

trainer = Trainer(model, config, train_loader=train_dl, valid_loader=test_dl)
trainer._load_checkpoint(config.num_epochs)
trainer.model.eval()

Using cache found in /Users/b1/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/b1/Library/Python/3.8/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/b1/Library/Python/3.8/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Files already downloaded and verified
Files already downloaded and verified


ResNet18(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

Посмортим предсказания претрейна

Замерим время

Посчитаем статистики

Посмотрим на графики врмени (ноды отсортированы по среднему). Время имеет логарифмическую шкалу!

Гиперпараметры:

In [7]:
imodel = GraphInterperterWithGumbelSoftmaxGamma(model.eval())

optimizer = torch.optim.Adam([imodel.gammas])
loss_fn = torch.nn.CrossEntropyLoss()

# lambd = 0
# times = torch.tensor(measurments['mean'], dtype=torch.float32)

Всё ещё аналогичная проблема :(

> `ValueError: Expected input batch_size (1) to match target batch_size (64).`

In [8]:
epoch_number = 0

EPOCHS = 1

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print(f'EPOCH {epoch_number}:')

    imodel.train()

    epoch_history = []
    for i, (X, y) in tqdm(enumerate(train_dl), 'training'):
        optimizer.zero_grad()
        y_pred = imodel(X)
        loss = loss_fn(y_pred, y)# + lambd * imodel.gammas.dot(times)
        loss.backward(retain_graph=True)
        optimizer.step()

        epoch_history.append(loss.item())
    avg_loss = np.mean(epoch_history)

    imodel.eval()
    
    print('epoch trained')

    running_vloss = 0.0
    for i, (X, y) in tqdm(enumerate(test_dl), 'validating'):
        if X.shape[0] != 64:
            continue
        running_vloss += loss_fn(imodel(X), y)
    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 0:


training: 0it [00:00, ?it/s]


ValueError: Expected input batch_size (1) to match target batch_size (64).